In [ ]:
import yaml
import oemof.solph

from mtress._meta_model import Location

In [ ]:
cfg_string = """
locations:
  LocationA:
    energy_cost:
      electricity:
        demand_rate: 15000  # €/MW
        working: 300  # €/MWh
      gas:
        working_price: 95  # €/MWh
        demand_rate: 5.5  # €/MW
      wood_pellet: 300  # €/MWh
    
    meteorology:
      temp_air: meteo.csv:temp_air
      temp_soil: meteo.csv:temp_soil
    
    energy_carriers:
      Heat: # class name
        reference_temperature: 10 # °C
        temperature_levels:
          - 35. # °C
          - 60. # °C
          - 80. # °C
      Electricity:
        grid_connection: True

    demand:
      electricity:
        - level: 400.
          values: demand.csv:lv_demand
        - level: 20000.
          values: demand.csv:mv_demand
      heat:
        - flow: 50.
          return: 30.
          values: demand.csv:heating
        - flow: 65.
          return: 40.
          values: demand.csv:dhw_heat_exchanger
        - flow: 65.
          return: 10.
          values: demand.csv:dhw_fresh_water_refill

"""

In [ ]:
cfg = yaml.safe_load(cfg_string)

In [ ]:
print(cfg)

In [ ]:

es = oemof.solph.EnergySystem()

In [ ]:
l = Location(
    name="LocationA",
    energy_system=es,
    meta_model=None,
    config=cfg["locations"]["LocationA"]
)

In [ ]:
import graphviz
import re

# Define shapes for the component types
SHAPES = {oemof.solph.Source: 'trapezium',
          oemof.solph.Sink: 'invtrapezium',
          oemof.solph.Bus: 'ellipse',
          oemof.solph.Transformer: 'octagon',
          oemof.solph.GenericStorage: 'cylinder'}

def generate_graph(energysystem):
    """Generate graphviz graph from energysystem."""
    dot = graphviz.Digraph(format='png')
    nodes = {}

    for node in energysystem.nodes:
        # Replace invalid characters
        name = nodes[node] = re.sub(r'[^a-zA-Z0-9]+', '__', node.label)

        dot.node(name, label=node.label, shape=SHAPES.get(type(node), 'rectangle'))

    for node in energysystem.nodes:
        for output in node.outputs:
            dot.edge(nodes[node], nodes[output])
    
    return dot


In [ ]:
generate_graph(l._energy_system)